In [1]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os, datetime
from dotenv import load_dotenv
from PIL import Image
import io
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

load_dotenv()

def upload_file_to_blob(blob_service_client, container_name, file_path, blob_name):
    try:
        # Create the container if it doesn't already exist
        container_client = blob_service_client.get_container_client(container_name)
        if not container_client.exists():
            container_client = blob_service_client.create_container(container_name)

        # Create a blob client using the local file name as the name for the blob
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

        # Upload the file
        with open(file_path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)
        print("File was uploaded successfully to Blob storage.")

    except Exception as e:
        print(f"An error occurred: {e}")

def get_blob_service_client(connection_string):
    return BlobServiceClient.from_connection_string(connection_string)

connection_string = os.getenv("CSTR__BLOB__VISINSP")

# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = get_blob_service_client(connection_string)

container_name = "image-uploads"


In [ ]:
if 1 == 1:
    import tempfile
    import tensorflow as tf
    blob_client = blob_service_client.get_blob_client(container="config", blob="trained_model.keras")

    with tempfile.NamedTemporaryFile(suffix='.keras', delete=True) as temp_file:
        # Download the blob content
        print("aaa")
        download_stream = blob_client.download_blob()
        print("bbb")
        temp_file.write(download_stream.readall())
        print("ccc")
        temp_file.flush()  # Ensure all data is written to disk
        print("ddd")
        # Seek to the start of the file to ensure tf.keras can read it from the beginning
        temp_file.seek(0)
        print("eee")
        # Load the model using TensorFlow/Keras
        model = tf.keras.models.load_model(temp_file.name)

In [14]:
# Create a temporary file manually
temp_dir = tempfile.gettempdir()
temp_file_path = os.path.join(temp_dir, "temp_model.keras")

try:
    # Download the blob to the temporary file
    with open(temp_file_path, "wb") as temp_file:
        download_stream = blob_client.download_blob()
        temp_file.write(download_stream.readall())
        temp_file.flush()

    # Load the model using TensorFlow/Keras
    model = tf.keras.models.load_model(temp_file_path)
    print("Model loaded successfully")
finally:
    # Clean up the temporary file
    if os.path.exists(temp_file_path):
        os.remove(temp_file_path)

Model loaded successfully


In [ ]:
with tempfile.NamedTemporaryFile(suffix='.keras', delete=True) as temp_file:
    temp_file.write(b"aaaaaaa")
    temp_file.seek(0)
    b = temp_file.read()
    print(temp_file.name)

print(b)

In [ ]:
if 1 == 2:
    file_path = r"C:\Users\oliver.koehn\Documents\gitProjects\qualityInspection\.gitignore"
    blob_name = "dummytest123"
    upload_file_to_blob(blob_service_client, container_name, file_path, blob_name)

In [3]:
base_path_local_files = r"C:\Users\Rebekka\gitProjects\visInspection\in\samples"
l__ok_images = [998]
for n in l__ok_images:
    fn_original = f"cast_ok_0_{n}.jpeg"
    fn_blob = fn_original.replace(".jpeg", f"__{datetime.datetime.now().strftime('%Y%m%d%H%M%S%f')}.jpeg")
    fp_local = os.path.join(base_path_local_files, fn_original)
    fp_blob = f"{datetime.datetime.now().strftime('%Y')}/{datetime.datetime.now().strftime('%m')}/{datetime.datetime.now().strftime('%d')}/{fn_blob}"
    print(f"UPLOAD LOCAL FILE {fn_original} to BLOB AS {fn_blob}")
    print(f"fp_local: {fp_local}")
    print(f"fp_blob: {fp_blob}")
    upload_file_to_blob(blob_service_client, container_name, fp_local, fp_blob)

UPLOAD LOCAL FILE cast_ok_0_998.jpeg to BLOB AS cast_ok_0_998__20240531173844585901.jpeg
fp_local: C:\Users\Rebekka\gitProjects\visInspection\in\samples\cast_ok_0_998.jpeg
fp_blob: 2024/05/31/cast_ok_0_998__20240531173844585901.jpeg
File was uploaded successfully to Blob storage.


In [ ]:
def read_image_from_blob(blob_service_client, container_name, blob):
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob)
    stream = blob_client.download_blob()
    return Image.open(io.BytesIO(stream.readall())) # image.show()

def format_image_for_model(image):
    image = image.convert('RGB')
    image = image.resize((200, 200))
    image_array = img_to_array(image)
    image_array = image_array * (1./255)
    return np.expand_dims(image_array, axis=0)

def eval_img_from_blob(loaded_model, blob_service_client, container_name, blob):
    image = read_image_from_blob(blob_service_client, container_name, blob)
    img_array = format_image_for_model(image)
    result = loaded_model.predict(img_array)
    prob__ok = result[0][1]
    #j__db = {"FP__BLOB" : blob, "CONTAINER_NAME" : container_name, "PROBABILITY_OK": prob__ok}

    return prob__ok

eval_img_from_blob(loaded_model, blob_service_client, container_name, "2024/05/21/cast_ok_0_31__20240521155649383512.jpeg")